
### Solow Model with Human Capital: Key Equations

1. **Production Function:**
   The extended Cobb-Douglas production function including human capital is given by:
   $$
   Y_t = K_t^\alpha H_t^\phi (A_t L_t)^{1-\alpha-\phi}
   $$

2. **Capital Accumulation Equations:**
   - **Physical Capital:**
     $$
     K_{t+1} = s_K Y_t + (1 - \delta) K_t
     $$
   - **Human Capital:**
     $$
     H_{t+1} = s_H Y_t + (1 - \delta) H_t
     $$
   where s_K and s_H are the savings rates for physical and human capital, and delta is the common depreciation rate for both types of capital.

3. **Per Worker Terms:**
   Transforming these equations into per effective worker terms involves normalizing by the effective labor:
   - **Capital per effective worker:**
     $$
     \tilde{k}_{t+1} = \frac{s_K \tilde{y}_t + (1-\delta) \tilde{k}_t}{(1+n)(1+g)}
     $$
   - **Human capital per effective worker:**
     $$
     \tilde{h}_{t+1} = \frac{s_H \tilde{y}_t + (1-\delta) \tilde{h}_t}{(1+n)(1+g)}
     $$
   where n is the growth rate of labor and g is the growth rate of technology.

4. **Steady State Conditions:**
   At the steady state, the capital stocks per effective worker stop changing:
   $$
   \tilde{k}_{t+1} = \tilde{k}_t = \tilde{k}^*
   $$
   $$
   \tilde{h}_{t+1} = \tilde{h}_t = \tilde{h}^*
   $$

5. **Steady State for Capital and Human capital:**
The steady state values for capital per effective worker and human capital per effective worker are given by the following equations:

- **Capital per effective worker:**
  $$
  \tilde{k}^* = \left(\frac{s_K^{1-\phi} s_H^\phi}{n+g+\delta+n\delta}\right)^{\frac{1}{1-\alpha-\phi}}
  $$

- **Human capital per effective worker:**
  $$
  \tilde{h}^* = \left(\frac{s_K^\alpha s_H^{1-\alpha}}{n+g+\delta+n\delta}\right)^{\frac{1}{1-\alpha-\phi}}
  $$

6. **Steady state for output per effective worker:**
The steady state output per effective worker (\( \tilde{y}^* \)) is given by the following equation:

$$
\tilde{y}^* = (\tilde{k}^*)^\alpha (\tilde{h}^*)^\phi
$$

Using the values for capital per effective worker and human capital per effective worker derived previously:

$$
\tilde{y}^* = \left( \left( \frac{s_K^{1-\phi} s_H^\phi}{n+g+\delta+n\delta} \right)^{\frac{\alpha}{1-\alpha-\phi}} \right) \left( \left( \frac{s_K^\alpha s_H^{1-\alpha}}{n+g+\delta+n\delta} \right)^{\frac{\phi}{1-\alpha-\phi}} \right)
$$

Expanding this, we get:

$$
\tilde{y}^* = \left( \frac{s_K}{n+g+\delta+n\delta} \right)^{\frac{\alpha}{1-\alpha-\phi}} \left( \frac{s_H}{n+g+\delta+n\delta} \right)^{\frac{\phi}{1-\alpha-\phi}}
$$


In [3]:
def steady_state(sK, sH, delta, alpha, phi, n, g):
    # Solving for k_tilde_star and h_tilde_star from steady state equations
    A = (sK / (n + g + delta)) ** (1 / (1 - alpha - phi))
    B = (sH / (n + g + delta)) ** (1 / (1 - alpha - phi))
    k_tilde_star = A ** (phi / (alpha * (1 - phi - alpha)))
    h_tilde_star = B ** (alpha / (phi * (1 - phi - alpha)))
    
    # Calculate y_tilde_star using the Cobb-Douglas production function
    y_tilde_star = k_tilde_star ** alpha * h_tilde_star ** phi
    
    return {'k_tilde_star': k_tilde_star, 'h_tilde_star': h_tilde_star, 'y_tilde_star': y_tilde_star}

# Example parameters
params = {
    'sK': 0.3,  # Physical capital savings rate
    'sH': 0.2,  # Human capital savings rate
    'delta': 0.05,  # Depreciation rate
    'alpha': 0.3,  # Output elasticity of physical capital
    'phi': 0.2,   # Output elasticity of human capital
    'n': 0.01,    # Population growth rate
    'g': 0.02     # Technological growth rate
}

# Calculate steady state values
steady_states = steady_state(**params)
print(steady_states)


{'k_tilde_star': 33.94298617754171, 'h_tilde_star': 244.1406250000001, 'y_tilde_star': 8.644761670141202}
